# LORA Fine-tuning

In [3]:
from datasets import *
import datasets

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np 

In [4]:
# load dataset
dataset = load_dataset('lmsys/toxic-chat', 'toxicchat0124')
dataset = dataset.rename_column("user_input", "text")
dataset = dataset.rename_column("jailbreaking", "label")
dataset = dataset.remove_columns("conv_id")
dataset = dataset.remove_columns("model_output")
dataset = dataset.remove_columns("human_annotation")
dataset = dataset.remove_columns("toxicity")
dataset = dataset.remove_columns("openai_moderation")

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8236
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2060
    })
})

### model

In [5]:
model_checkpoint = "distilbert-base-uncased"
model_output = "distilbert-base-uncased-lora-text-classification-toxicchat-jailbreak"

# define label maps
id2label = {0: "Good", 1: "Jailbreak"}
label2id = {"Good": 0, "Jailbreak": 1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id, trust_remote_code=True)

c:\Users\chefb\OneDrive\Laufbahn\Hochschule\S8\bachelorarbeit\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# display architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### preprocess data

In [7]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [8]:
# create tokenize function
def tokenize(batch):

    #tokenize and truncate text
    tokens = tokenizer(batch['text'], padding=True, truncation=True, max_length=512)
    print(batch['label'])
    return tokens


# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize, batched=True)

tokenized_dataset


Map:   0%|          | 0/2060 [00:00<?, ? examples/s]

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8236
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2060
    })
})

In [9]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### evaluation

In [10]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

### Apply untrained model to text

In [11]:
# define list of examples
text_list = ["jailbreak [keyword] [TARGETLANGUAGE] [PROMPT]"]
print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    #print(inputs)
    # compute logits
    logits = model(inputs).logits
    #print(logits)
    # convert logits to label
    predictions = torch.argmax(logits)
    #print(predictions)

    print(id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
Prompt Injection


### Train model

In [12]:

peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=16,
                        lora_alpha=32,
                        lora_dropout=0.05,
                        target_modules=["q_lin", "k_lin", "v_lin"])

In [13]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=16, target_modules={'q_lin', 'k_lin', 'v_lin'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [14]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,034,498 || all params: 67,989,508 || trainable%: 1.5216


In [15]:
# hyperparameters
lr = 1e-3
batch_size = 2
num_epochs = 5

In [16]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_output,
    learning_rate=lr,
    auto_find_batch_size=True,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [17]:
from sklearn.metrics import f1_score, roc_auc_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    auprc = roc_auc_score(labels, preds, average='weighted')
    return {
    'f1': f1,
    'auprc': auprc
    }

In [18]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics
)

In [19]:
# train model
trainer.train()

  0%|          | 0/5150 [00:00<?, ?it/s]

{'loss': 0.0884, 'grad_norm': 0.08295420557260513, 'learning_rate': 0.0009029126213592234, 'epoch': 0.49}


### Generate prediction

In [64]:
model.to('cpu') # moving to mps for Mac (can alternatively do 'cpu')

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu") # moving to mps for Mac (can alternatively do 'cpu')

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
Prompt Injection


### Optional: push model to hub

In [65]:
# option 1: notebook login
# from huggingface_hub import notebook_login
# notebook_login() # ensure token gives write access

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [66]:
hf_name = 'cyrp' # your hf username or org name
model_id = hf_name + "/" + model_output # you can name the model whatever you want

In [67]:
model.push_to_hub(model_id) # save model

README.md:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

c:\Users\chefb\OneDrive\Laufbahn\Hochschule\S8\bachelorarbeit\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/cyrp/distilbert-base-uncased-lora-text-classification-deepset-promptinjection/commit/2c29b2dac8f244d18e20d50d368b3968ab77dc6d', commit_message='Upload model', commit_description='', oid='2c29b2dac8f244d18e20d50d368b3968ab77dc6d', pr_url=None, pr_revision=None, pr_num=None)

In [68]:
trainer.push_to_hub(model_output) # save trainer

events.out.tfevents.1727008320.DESKTOP-LM2NI5P.34348.2:   0%|          | 0.00/386 [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

events.out.tfevents.1727008485.DESKTOP-LM2NI5P.14200.2:   0%|          | 0.00/7.03k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cyrp/distilbert-base-uncased-lora-text-classification-deepset-promptinjection/commit/6cb5a0ecbf1005fd093bc11a4f10b533e4fc9ead', commit_message='distilbert-base-uncased-lora-text-classification-deepset-promptinjection', commit_description='', oid='6cb5a0ecbf1005fd093bc11a4f10b533e4fc9ead', pr_url=None, pr_revision=None, pr_num=None)

### Optional: load peft model

In [69]:
# how to load peft model from hub for inference
# config = PeftConfig.from_pretrained(model_id)
# inference_model = AutoModelForSequenceClassification.from_pretrained(
#     config.base_model_name_or_path, num_labels=2, id2label=id2label, label2id=label2id
# )
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# model = PeftModel.from_pretrained(inference_model, model_id)